In [1]:
import numpy as np 
import pandas as pd
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model
import keras

In [5]:
english_texts=[]
russian_texts=[]
english_character=[]
russian_character=[]
with open("./rus.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:30000]:
    english_text,russian_text,_=line.split("\t")
    english_texts.append(english_text)
    russian_text = "\t" + russian_text + "\n"
    russian_texts.append(russian_text)

In [6]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()

In [7]:
for j in russian_texts:
    for c in j:
        if c not in russian_character:
            russian_character.append(c)
            russian_character.sort()

In [8]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [9]:
russian_d={}
for i in range(len(russian_character)):
    russian_d[russian_character[i]]=i

In [10]:
english_encoder_tokens = len(english_character)
russian_decoder_tokens = len(russian_character)

In [11]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)

In [12]:
max_decoder_seq_length=0
for i in russian_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [13]:
encoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)

In [14]:
encoder_input_data=np.array(encoder_input_data)

In [15]:
decoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=0
    while m<k:
        for char in russian_texts[bb][m]:
            for i in range(len(russian_character)):
                if russian_d[char]==i:
                    a.append(1)
                else:
                    a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [16]:
decoder_input_data=np.array(decoder_input_data)

In [17]:
decoder_target_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=1
    while m<k:
        for char in russian_texts[bb][m]:
                           for i in range(len(russian_character)):
                                            if russian_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [18]:
decoder_target_data=np.array(decoder_target_data)

In [19]:
model = keras.models.load_model("engtorussian.h5")
encoder_inputs = model.input[0]  # input_1
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [20]:
latent_dim = 256 

In [21]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,),)
decoder_state_input_c = keras.Input(shape=(latent_dim,),)
decoder_state_input_h1 = Input(shape=(latent_dim,),)
decoder_state_input_c1 = Input(shape=(latent_dim,),)
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [22]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(russian_character)):
    reverse_target_char_index[i]=russian_character[i]

In [23]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(russian_character)))
    target_seq[0, 0, russian_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(russian_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent

In [113]:
english='He will not go.'

In [114]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

In [115]:
inpu=np.array(inpu)

In [116]:
inpu

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]])

In [117]:
d=decode_sequence(inpu)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [118]:
d

'Он не поедет.\n'